Ref: https://github.com/tungk/OED

In [1]:
import os
import random

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tqdm.notebook import tqdm
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, LSTM, Reshape, Dropout, GaussianNoise, Concatenate, Lambda, RepeatVector

# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
def SKip_AE(seq_length, dim, N_RES=3, N_LAYERS=1, BATCH_SIZE=128):
    tf.keras.backend.clear_session()
    
    en_inputs = []
    shared_latents = []
    for n in range(N_RES):
        selected_t = [t for t in range(0, seq_length, 2**n)]
        en_input = Input(shape=[len(selected_t), dim])
        en_inputs.append(en_input)

    de_outputs = []
    for n in range(N_RES):
        for l in range(N_LAYERS):
            X = en_inputs[n] if l == 0 else X
            X = GaussianNoise(0.5)(X) if l == 0 else X
            X = LSTM(64, return_sequences=False if l + 1 == N_LAYERS else True)(X)
        h = Dense(32, activation='relu', kernel_regularizer='l1')(X)
        shared_latents.append(h)

    for n in range(N_RES):
        selected_t = [t for t in range(0, seq_length, 2**n)]
        for l in range(N_LAYERS):
            if l == 0:
                X = Concatenate()(shared_latents)
                X = Dense(128, activation='relu')(X)
                X = Concatenate()([X, shared_latents[n]])
                X = RepeatVector(len(selected_t))(X)
            X = LSTM(64, return_sequences=False if l + 1 == N_LAYERS else True)(X)
        rec_x = Dense(len(selected_t) * dim)(X)
        rec_x = Reshape([len(selected_t), dim])(rec_x)
        de_outputs.append(rec_x)

    model = Model(inputs=en_inputs, outputs=de_outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")

    return model

### Yahoo S5

In [4]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [5]:
for loader in [load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4]:
    datasets = loader(128, 32)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = SKip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)
    
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

yahoo_A1 0.5714285224489823 0.1666666623148149 0.1666666583333337
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.9999998500000123 0.49999990000001493 0.9999998888889001
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.46153841893491365 0.14999999350000026 0.49999997619047704
yahoo_A1 0.6666666133333363 0.6666666005092657 0.6249999718750009
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.7999999200000041 0.6767856563734693 0.6666666311111126
yahoo_A1 0.33333330000000166 0.055555557160493235 0.49999994375000567
yahoo_A1 0.8888888296296326 0.652777698252327 0.6666665777777875
yahoo_A1 0.6666666133333363 0.08333334749999795 -0.0
yahoo_A1 0.9999999000000052 0.7499998875000143 0.9999999375000035
yahoo_A1 0.18181816198347142 0.0 0.0
yahoo_A1 0.46153841893491365 0.0 0.0
yahoo_A1 0.9999998500000123 0.49999990000001493 0.9999998888889002
yahoo_A1 0.6666666133333363 0.15476190589592742 -0.0
yahoo_A1 0.6666666133333363 0.08333334620204845 -0.0
yahoo_A1 0.749999943750003 0.10000001

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

yahoo_A3 0.4999999562500026 0.0 0.0
yahoo_A3 0.5882352456747433 0.06250001071706508 -0.0
yahoo_A3 0.736842050969532 0.0 0.0
yahoo_A3 0.4999999562500026 0.0 0.0
yahoo_A3 0.8571428000000028 1.025883832329092e-09 0.0
yahoo_A3 0.4999999562500026 6.464645887944386e-09 0.0
yahoo_A3 0.736842050969532 0.0833333493093114 -0.0
yahoo_A3 0.6666666148148177 0.0 0.0
yahoo_A3 0.799999944000003 0.0 0.0
yahoo_A3 0.0 0.0 0.0
yahoo_A3 0.9565216809073751 0.25000002354554557 -0.0
yahoo_A3 0.9565216809073751 0.0 0.0
yahoo_A3 0.799999944000003 0.0 0.0
yahoo_A3 0.28571425714285853 0.04545454693526121 -0.0
yahoo_A3 0.736842050969532 5.447330150360656e-09 0.0
yahoo_A3 0.15384613727810673 0.0 0.0
yahoo_A3 0.9090908512396723 0.0 0.0
yahoo_A3 0.4999999562500026 0.0 0.0
yahoo_A3 0.4999999562500026 6.464645887944386e-09 0.0
yahoo_A3 0.5882352456747433 0.0 0.0
yahoo_A3 0.28571425714285853 6.439392944080749e-09 0.0
yahoo_A3 0.28571425714285853 0.0 0.0
yahoo_A3 0.736842050969532 3.0862191832992026e-09 0.0
yahoo_A3 0.95

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

yahoo_A4 0.6666666148148177 0.0 0.0
yahoo_A4 0.0 0.0 0.0
yahoo_A4 0.4999999562500026 0.05555556362513912 -0.0
yahoo_A4 0.6666666148148177 0.0 0.0
yahoo_A4 0.4999999562500026 0.05555556362513912 -0.0
yahoo_A4 0.9090908512396723 0.16666669088383523 -0.0
yahoo_A4 0.736842050969532 0.08333334928826763 -0.0
yahoo_A4 0.4999999562500026 0.0 0.0
yahoo_A4 0.0 0.0 0.0
yahoo_A4 0.5882352456747433 0.0 0.0
yahoo_A4 0.0 0.0 -0.0
yahoo_A4 0.9090908512396723 7.709234992875049e-09 0.0
yahoo_A4 0.736842050969532 0.0 0.0
yahoo_A4 0.9090908512396723 0.0 0.0
yahoo_A4 0.6666666148148177 0.0 0.0
yahoo_A4 0.736842050969532 0.08333334928826763 0.0
yahoo_A4 0.0 0.0 -0.0
yahoo_A4 0.3999999626666689 0.0 0.0
yahoo_A4 0.15384613727810673 0.04166666215277823 -0.0
yahoo_A4 0.4999999562500026 0.0 0.0
yahoo_A4 0.9090908512396723 0.0 0.0
yahoo_A4 0.8571428000000028 0.0 0.0
yahoo_A4 0.9999999416666693 0.0 0.0
yahoo_A4 0.9565216809073751 0.0 0.0
yahoo_A4 0.4999999562500026 0.0 0.0
yahoo_A4 0.736842050969532 0.083333349288

In [6]:
yahoo_results = pd.DataFrame(total_scores)

In [7]:
yahoo_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
yahoo_A1,0.440259,0.139369,0.197554
yahoo_A2,0.000000,0.000000,0.000000
yahoo_A3,0.646359,0.031808,0.008333
yahoo_A4,0.557455,0.022836,0.013472


### NASA

In [8]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [9]:
for loader in [load_nasa]:
    datasets = loader(100, 100)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = SKip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)
        
        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

D1 0.2658442980987866 0.067217603160146 0.3091180118255026
D2 0.29898400693628724 0.14465048756846324 0.5127462431770149


In [10]:
nasa_results = pd.DataFrame(total_scores)

In [11]:
nasa_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.265844,0.067218,0.309118
D2,0.298984,0.144650,0.512746


### SMD

In [12]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [13]:
for loader in [load_smd]:
    datasets = loader(100, 100)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = SKip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)
      
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

smd 0.21943571700356884 0.06161971811210573 0.49999999837062536
smd 0.1269841149911827 0.0 0.0
smd 0.29090904357025527 0.1416417253237693 0.6735144273602699
smd 0.23809518741497632 0.12840560299529974 0.6359446968989098
smd 0.07239818300198662 0.021591084692673755 0.41983779857178904
smd 0.4418604270807075 0.20206575241995145 0.48936543747018696
smd 0.2762430617502555 0.13016062364296743 0.5186869292590344
smd 0.3888888424768571 0.17551693294379772 0.6440913141010485
smd 0.18461536771597778 0.07527671689747409 0.3908236113437713
smd 0.35233157550538524 0.11307376475794409 0.6614495184590106
smd 0.1044176607022476 0.027542372657821033 0.49999999592963096
smd 0.2710622474312033 0.11658154114276005 0.4155717817859286
smd 0.3513513185536917 0.16390503692656946 0.6614079516038013
smd 0.08026755077012628 0.019305019171533672 0.5472727216797796
smd 0.18461536771597778 0.06950003409522049 0.34265900133253513
smd 0.02499999751041683 0.006329113710409663 0.4999999831196587
smd 0.1719745064647667

In [14]:
smd_results = pd.DataFrame(total_scores)

In [15]:
smd_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
smd,0.187487,0.061142,0.450517
